# 🚀 Demonstração do Sistema de Microserviços para Condomínios

## 📋 Visão Geral

Este notebook demonstra como interagir com nosso sistema de microserviços para gestão de condomínios, incluindo:

- **MSResident** (Porta 8081) - Gerenciamento de moradores
- **MSFinancialManagement** (Porta 8082) - Gestão financeira e upload de arquivos
- **MSProprietary** (Porta 8083) - Administração de proprietários

## 🎯 Objetivos da Demonstração

1. Verificar que todos os serviços estão funcionando
2. Testar operações CRUD em cada microserviço
3. Demonstrar a independência dos serviços
4. Mostrar tratamento de erros e resiliência
5. Apresentar cenários completos de uso

---

In [1]:
# Import Required Libraries
import requests
import json
import pandas as pd
from datetime import datetime
import time
import warnings
warnings.filterwarnings('ignore')

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


In [2]:
# Define Base URLs and Headers
BASE_URLS = {
    'resident': 'http://localhost:8081',
    'financial': 'http://localhost:8082', 
    'proprietary': 'http://localhost:8083'
}

HEADERS = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}

TIMEOUT = 5  # seconds

print("🔧 Configuração dos microserviços:")
for service, url in BASE_URLS.items():
    print(f"  {service.upper()}: {url}")
    
print(f"\n⏱️ Timeout configurado para: {TIMEOUT} segundos")

🔧 Configuração dos microserviços:
  RESIDENT: http://localhost:8081
  FINANCIAL: http://localhost:8082
  PROPRIETARY: http://localhost:8083

⏱️ Timeout configurado para: 5 segundos


In [3]:
# Test Service Health Status
def check_service_health(service_name, base_url):
    """Verifica se um serviço está saudável"""
    try:
        response = requests.get(f"{base_url}/health", timeout=TIMEOUT)
        if response.status_code == 200:
            health_data = response.json()
            return True, health_data
        else:
            return False, f"Status Code: {response.status_code}"
    except requests.exceptions.RequestException as e:
        return False, str(e)

print("🩺 Verificando saúde dos microserviços...\n")

service_status = {}
for service, url in BASE_URLS.items():
    print(f"🔍 Testando {service.upper()}...")
    is_healthy, result = check_service_health(service, url)
    service_status[service] = is_healthy
    
    if is_healthy:
        print(f"✅ {service.upper()} está funcionando!")
        print(f"   Status: {result.get('status', 'N/A')}")
        print(f"   Serviço: {result.get('service', 'N/A')}")
        print(f"   Versão: {result.get('version', 'N/A')}")
    else:
        print(f"❌ {service.upper()} não está respondendo: {result}")
    print()

# Resumo final
healthy_services = sum(service_status.values())
total_services = len(service_status)
print(f"📊 Resumo: {healthy_services}/{total_services} serviços funcionando")

if healthy_services == total_services:
    print("🎉 Todos os serviços estão saudáveis!")
else:
    print("⚠️ Alguns serviços podem precisar de atenção")

🩺 Verificando saúde dos microserviços...

🔍 Testando RESIDENT...
✅ RESIDENT está funcionando!
   Status: UP
   Serviço: MSResident
   Versão: 1.0.0

🔍 Testando FINANCIAL...
✅ FINANCIAL está funcionando!
   Status: UP
   Serviço: MSFinancialManagement
   Versão: 1.0.0

🔍 Testando PROPRIETARY...
✅ PROPRIETARY está funcionando!
   Status: UP
   Serviço: MSProprietary
   Versão: 1.0.0

📊 Resumo: 3/3 serviços funcionando
🎉 Todos os serviços estão saudáveis!


In [4]:
# Discover Correct API Endpoints
def test_endpoint(base_url, endpoint, method='GET'):
    """Testa um endpoint específico"""
    url = f"{base_url}{endpoint}"
    try:
        if method.upper() == 'GET':
            response = requests.get(url, timeout=TIMEOUT, headers=HEADERS)
        elif method.upper() == 'POST':
            response = requests.post(url, timeout=TIMEOUT, headers=HEADERS, json={})
        else:
            return False, "Método não suportado"
            
        return response.status_code != 404, response.status_code
    except requests.exceptions.RequestException as e:
        return False, str(e)

print("🔍 Descobrindo endpoints corretos dos microserviços...\n")

# Endpoints para testar
endpoints_to_test = {
    'resident': [
        '/residents',
        '/api/residents', 
        '/resident',
        '/api/resident'
    ],
    'financial': [
        '/files/',
        '/api/files',
        '/upload',
        '/api/upload'
    ],
    'proprietary': [
        '/proprietaries',
        '/api/proprietaries',
        '/proprietary',
        '/api/proprietary'
    ]
}

discovered_endpoints = {}

for service, endpoints in endpoints_to_test.items():
    print(f"🔍 Testando {service.upper()}:")
    discovered_endpoints[service] = []
    
    for endpoint in endpoints:
        is_valid, status = test_endpoint(BASE_URLS[service], endpoint)
        if is_valid:
            print(f"  ✅ {endpoint} - Status: {status}")
            discovered_endpoints[service].append(endpoint)
        else:
            print(f"  ❌ {endpoint} - Status: {status}")
    
    print(f"  📋 Endpoints válidos: {len(discovered_endpoints[service])}")
    print()

print("📊 Resumo dos endpoints descobertos:")
for service, endpoints in discovered_endpoints.items():
    print(f"  {service.upper()}: {endpoints}")
    
# Definir endpoints principais baseados na descoberta
MAIN_ENDPOINTS = {
    'resident': '/residents',
    'financial': '/files/',
    'proprietary': '/proprietaries'
}

print(f"\n🎯 Endpoints principais selecionados: {MAIN_ENDPOINTS}")

🔍 Descobrindo endpoints corretos dos microserviços...

🔍 Testando RESIDENT:
  ✅ /residents - Status: 500
  ❌ /api/residents - Status: 404
  ❌ /resident - Status: 404
  ❌ /api/resident - Status: 404
  📋 Endpoints válidos: 1

🔍 Testando FINANCIAL:
  ✅ /files/ - Status: 500
  ❌ /api/files - Status: 404
  ❌ /upload - Status: 404
  ❌ /api/upload - Status: 404
  📋 Endpoints válidos: 1

🔍 Testando PROPRIETARY:
  ✅ /proprietaries - Status: 200
  ❌ /api/proprietaries - Status: 404
  ❌ /proprietary - Status: 404
  ❌ /api/proprietary - Status: 404
  📋 Endpoints válidos: 1

📊 Resumo dos endpoints descobertos:
  RESIDENT: ['/residents']
  FINANCIAL: ['/files/']
  PROPRIETARY: ['/proprietaries']

🎯 Endpoints principais selecionados: {'resident': '/residents', 'financial': '/files/', 'proprietary': '/proprietaries'}


In [ ]:
# Implement Residents API Calls
class ResidentAPI:
    def __init__(self, base_url):
        self.base_url = base_url
        self.endpoint = '/residents'
    
    def get_all_residents(self):
        """Busca todos os moradores"""
        try:
            response = requests.get(f"{self.base_url}{self.endpoint}", 
                                  timeout=TIMEOUT, headers=HEADERS)
            return response.status_code == 200, response.json() if response.status_code == 200 else response.text
        except Exception as e:
            return False, str(e)
    
    def get_resident(self, resident_id):
        """Busca um morador específico"""
        try:
            response = requests.get(f"{self.base_url}{self.endpoint}/{resident_id}", 
                                  timeout=TIMEOUT, headers=HEADERS)
            return response.status_code == 200, response.json() if response.status_code == 200 else response.text
        except Exception as e:
            return False, str(e)
    
    def create_resident(self, resident_data):
        """Cria um novo morador"""
        try:
            response = requests.post(f"{self.base_url}{self.endpoint}", 
                                   json=resident_data, timeout=TIMEOUT, headers=HEADERS)
            return response.status_code in [200, 201], response.json() if response.status_code in [200, 201] else response.text
        except Exception as e:
            return False, str(e)
    
    def update_resident(self, resident_id, resident_data):
        """Atualiza um morador existente"""
        try:
            response = requests.put(f"{self.base_url}{self.endpoint}/{resident_id}", 
                                  json=resident_data, timeout=TIMEOUT, headers=HEADERS)
            return response.status_code == 200, response.json() if response.status_code == 200 else response.text
        except Exception as e:
            return False, str(e)
    
    def delete_resident(self, resident_id):
        """Remove um morador"""
        try:
            response = requests.delete(f"{self.base_url}{self.endpoint}/{resident_id}", 
                                     timeout=TIMEOUT, headers=HEADERS)
            return response.status_code in [200, 204], "Morador removido com sucesso" if response.status_code in [200, 204] else response.text
        except Exception as e:
            return False, str(e)
    
    # MESSAGE SCHEDULING FEATURES
    def schedule_message(self, message_data):
        """Agenda uma mensagem para envio"""
        try:
            response = requests.post(f"{self.base_url}/messages/schedule", 
                                   json=message_data, timeout=TIMEOUT, headers=HEADERS)
            return response.status_code in [200, 201], response.json() if response.status_code in [200, 201] else response.text
        except Exception as e:
            return False, str(e)
    
    def get_scheduled_messages(self):
        """Busca todas as mensagens agendadas"""
        try:
            response = requests.get(f"{self.base_url}/messages/scheduled", 
                                  timeout=TIMEOUT, headers=HEADERS)
            return response.status_code == 200, response.json() if response.status_code == 200 else response.text
        except Exception as e:
            return False, str(e)
    
    def get_scheduled_message(self, message_id):
        """Busca uma mensagem agendada específica"""
        try:
            response = requests.get(f"{self.base_url}/messages/scheduled/{message_id}", 
                                  timeout=TIMEOUT, headers=HEADERS)
            return response.status_code == 200, response.json() if response.status_code == 200 else response.text
        except Exception as e:
            return False, str(e)
    
    def cancel_scheduled_message(self, message_id):
        """Cancela uma mensagem agendada"""
        try:
            response = requests.delete(f"{self.base_url}/messages/scheduled/{message_id}", 
                                     timeout=TIMEOUT, headers=HEADERS)
            return response.status_code in [200, 204], "Mensagem cancelada com sucesso" if response.status_code in [200, 204] else response.text
        except Exception as e:
            return False, str(e)

# Inicializar cliente da API de Moradores
resident_api = ResidentAPI(BASE_URLS['resident'])

print("🏠 Cliente da API de Moradores inicializado!")
print(f"   Base URL: {BASE_URLS['resident']}")
print(f"   Endpoint: {resident_api.endpoint}")
print("\n📋 Funções disponíveis:")
print("   - get_all_residents()")
print("   - get_resident(id)")
print("   - create_resident(data)")
print("   - update_resident(id, data)")
print("   - delete_resident(id)")
print("\n📅 Funções de agendamento de mensagens:")
print("   - schedule_message(data)")
print("   - get_scheduled_messages()")
print("   - get_scheduled_message(id)")
print("   - cancel_scheduled_message(id)")

🏠 Cliente da API de Moradores inicializado!
   Base URL: http://localhost:8081
   Endpoint: /residents

📋 Funções disponíveis:
   - get_all_residents()
   - get_resident(id)
   - create_resident(data)
   - update_resident(id, data)
   - delete_resident(id)


In [ ]:
# 📅 AGENDAMENTO DE MENSAGENS - EXEMPLOS PRÁTICOS
print("📅 DEMONSTRAÇÃO DE AGENDAMENTO DE MENSAGENS")
print("=" * 50)
print()

# Verificar se o sistema de mensagens está funcionando
print("1️⃣ Verificando saúde do sistema de mensagens:")
print("-" * 45)
success, result = resident_api.check_message_system_health()
format_response(success, result, "Status do sistema de mensagens")

# Exemplo 1: Agendar mensagem rápida (5 minutos)
print("2️⃣ Agendando mensagem rápida:")
print("-" * 35)

message_data = {
    "title": "Aviso Importante - Manutenção",
    "content": "Haverá manutenção do sistema elétrico amanhã das 8h às 12h. Por favor, se programem.",
    "recipient_type": "ALL",
    "minutes_from_now": 2,  # 2 minutos para teste
    "sender_name": "Administração",
    "message_type": "WARNING"
}

success, result = resident_api.schedule_quick_message(**message_data)
format_response(success, result, "Agendar mensagem rápida")

# Exemplo 2: Agendar mensagem com data específica
print("3️⃣ Agendando mensagem com data específica:")
print("-" * 45)

from datetime import datetime, timedelta

# Agendar para 10 minutos no futuro
future_time = datetime.now() + timedelta(minutes=10)
scheduled_time_str = future_time.isoformat()

advanced_message = {
    "title": "Reunião de Condomínio",
    "content": "Lembrete: Reunião de condomínio na próxima sexta-feira às 19h no salão de festas. Todos os moradores são convidados.",
    "recipient_type": "RESIDENTS",
    "scheduled_time": scheduled_time_str,
    "sender_name": "Síndico",
    "message_type": "INFO"
}

success, result = resident_api.schedule_message(**advanced_message)
format_response(success, result, "Agendar mensagem com data específica")

# Exemplo 3: Listar mensagens agendadas
print("4️⃣ Listando todas as mensagens agendadas:")
print("-" * 42)
success, result = resident_api.get_all_scheduled_messages()
format_response(success, result, "Listar mensagens agendadas")

# Exemplo 4: Contar mensagens pendentes
print("5️⃣ Contando mensagens pendentes:")
print("-" * 35)
success, result = resident_api.count_pending_messages()
format_response(success, result, "Contar mensagens pendentes")

# Exemplo 5: Agendar mensagem para morador específico
print("6️⃣ Agendando mensagem para morador específico:")
print("-" * 48)

specific_message = {
    "title": "Cobrança Pendente",
    "content": "Você possui uma cobrança pendente de R$ 150,00 referente à taxa de condomínio. Por favor, regularize sua situação.",
    "recipient_type": "SPECIFIC",
    "recipient_id": 1,  # ID do morador específico
    "minutes_from_now": 3,
    "sender_name": "Financeiro",
    "message_type": "PAYMENT_REMINDER"
}

success, result = resident_api.schedule_quick_message(
    specific_message["title"],
    specific_message["content"],
    specific_message["recipient_type"],
    specific_message["minutes_from_now"],
    specific_message["sender_name"],
    specific_message["message_type"]
)
format_response(success, result, "Agendar mensagem específica")

print()
print("📋 TIPOS DE DESTINATÁRIOS DISPONÍVEIS:")
print("-" * 40)
print("• ALL - Todos (moradores + proprietários)")
print("• RESIDENTS - Apenas moradores")
print("• PROPRIETARIES - Apenas proprietários") 
print("• SPECIFIC - Destinatário específico (requer recipient_id)")
print()
print("📋 TIPOS DE MENSAGEM DISPONÍVEIS:")
print("-" * 35)
print("• INFO - Informação geral")
print("• WARNING - Aviso importante")
print("• ALERT - Alerta urgente")
print("• PAYMENT_REMINDER - Lembrete de pagamento")
print()
print("✅ Sistema de agendamento de mensagens configurado!")

## 📅 Agendamento de Mensagens - API

### ✅ **Resposta Rápida: Como agendar uma mensagem**

Para agendar uma mensagem no sistema, você deve fazer uma requisição **POST** para:

```
POST http://localhost:8081/messages/schedule
```

### 📋 **Parâmetros da Chamada:**

#### **Agendamento Completo:**
```json
{
  "title": "Título da mensagem",
  "content": "Conteúdo da mensagem",
  "recipientType": "ALL|RESIDENTS|PROPRIETARIES|SPECIFIC",
  "scheduledTime": "2025-07-13T15:30:00",
  "senderName": "Nome do remetente",
  "messageType": "INFO|WARNING|ALERT|PAYMENT_REMINDER",
  "recipientId": 123 // apenas para recipientType = "SPECIFIC"
}
```

#### **Agendamento Rápido:**
```
POST http://localhost:8081/messages/schedule/quick?title=Titulo&content=Conteudo&minutesFromNow=5
```

### 🛠️ **Formas de agendar mensagens:**

#### 1. **Via CURL:**
```bash
# Agendamento rápido (5 minutos)
curl -X POST "http://localhost:8081/messages/schedule/quick" \
     -d "title=Aviso Importante" \
     -d "content=Manutenção programada para amanhã" \
     -d "recipientType=ALL" \
     -d "minutesFromNow=5"

# Agendamento com data específica
curl -X POST "http://localhost:8081/messages/schedule" \
     -H "Content-Type: application/json" \
     -d '{
       "title": "Reunião de Condomínio",
       "content": "Reunião na sexta às 19h",
       "recipientType": "ALL",
       "scheduledTime": "2025-07-15T19:00:00",
       "senderName": "Síndico",
       "messageType": "INFO"
     }'
```

#### 2. **Via Python:**
```python
# Agendamento rápido
resident_api.schedule_quick_message(
    title="Aviso Importante",
    content="Manutenção programada",
    recipient_type="ALL",
    minutes_from_now=5
)

# Agendamento com data específica
from datetime import datetime, timedelta
future_time = (datetime.now() + timedelta(hours=2)).isoformat()

resident_api.schedule_message(
    title="Reunião de Condomínio",
    content="Reunião na sexta às 19h",
    recipient_type="ALL",
    scheduled_time=future_time,
    sender_name="Síndico"
)
```

#### 3. **Via JavaScript:**
```javascript
// Agendamento rápido
fetch('http://localhost:8081/messages/schedule/quick', {
    method: 'POST',
    headers: {'Content-Type': 'application/x-www-form-urlencoded'},
    body: new URLSearchParams({
        title: 'Aviso Importante',
        content: 'Manutenção programada',
        recipientType: 'ALL',
        minutesFromNow: '5'
    })
});

// Agendamento completo
fetch('http://localhost:8081/messages/schedule', {
    method: 'POST',
    headers: {'Content-Type': 'application/json'},
    body: JSON.stringify({
        title: 'Reunião de Condomínio',
        content: 'Reunião na sexta às 19h',
        recipientType: 'ALL',
        scheduledTime: '2025-07-15T19:00:00',
        senderName: 'Síndico',
        messageType: 'INFO'
    })
});
```

### 📊 **Endpoints Adicionais:**

| Método | Endpoint | Descrição |
|--------|----------|-----------|
| `GET` | `/messages` | Lista todas as mensagens |
| `GET` | `/messages/{id}` | Busca mensagem por ID |
| `DELETE` | `/messages/{id}` | Cancela mensagem agendada |
| `GET` | `/messages/status/{status}` | Lista por status |
| `GET` | `/messages/count/pending` | Conta mensagens pendentes |
| `GET` | `/messages/overdue` | Lista mensagens vencidas |
| `GET` | `/messages/health` | Status do sistema |

### 🎯 **Tipos de Destinatários:**
- **ALL**: Todos (moradores + proprietários)
- **RESIDENTS**: Apenas moradores
- **PROPRIETARIES**: Apenas proprietários
- **SPECIFIC**: Destinatário específico (requer `recipientId`)

### 📝 **Tipos de Mensagem:**
- **INFO**: Informação geral
- **WARNING**: Aviso importante
- **ALERT**: Alerta urgente
- **PAYMENT_REMINDER**: Lembrete de pagamento

### ⏰ **Processamento Automático:**
O sistema verifica a cada **60 segundos** se há mensagens para enviar e as processa automaticamente.

---

In [ ]:
# Implement Financial Management API Calls
class FinancialAPI:
    def __init__(self, base_url):
        self.base_url = base_url
        self.endpoint = '/files'
    
    def get_files_page(self):
        """Acessa a página de listagem de arquivos"""
        try:
            response = requests.get(f"{self.base_url}{self.endpoint}/", 
                                  timeout=TIMEOUT)
            return response.status_code == 200, response.text if response.status_code == 200 else f"Status: {response.status_code}"
        except Exception as e:
            return False, str(e)
    
    def upload_file(self, file_path, payment_id):
        """Simula upload de arquivo"""
        try:
            # Para demonstração, vamos simular um upload
            files = {'file': ('demo.txt', 'Conteúdo de demonstração', 'text/plain')}
            data = {'paymentId': payment_id}
            
            response = requests.post(f"{self.base_url}{self.endpoint}/upload", 
                                   files=files, data=data, timeout=TIMEOUT)
            return response.status_code in [200, 201], response.text
        except Exception as e:
            return False, str(e)
    
    def upload_pdf_proof(self, pdf_file_path, payment_id):
        """Faz upload de um comprovante PDF para um pagamento específico"""
        try:
            import os
            
            # Verificar se o arquivo existe
            if not os.path.exists(pdf_file_path):
                return False, f"Arquivo não encontrado: {pdf_file_path}"
            
            # Preparar o arquivo para upload
            with open(pdf_file_path, 'rb') as file:
                files = {'file': (os.path.basename(pdf_file_path), file, 'application/pdf')}
                data = {'paymentId': payment_id}
                
                response = requests.post(f"{self.base_url}{self.endpoint}/upload", 
                                       files=files, data=data, timeout=TIMEOUT)
                
                return response.status_code in [200, 201], response.text
        except Exception as e:
            return False, str(e)
    
    def get_file(self, filename):
        """Busca um arquivo específico"""
        try:
            response = requests.get(f"{self.base_url}{self.endpoint}/files/{filename}", 
                                  timeout=TIMEOUT)
            return response.status_code == 200, response.text if response.status_code == 200 else f"Status: {response.status_code}"
        except Exception as e:
            return False, str(e)
    
    def check_health_info(self):
        """Verifica informações detalhadas de saúde"""
        try:
            response = requests.get(f"{self.base_url}/health/info", 
                                  timeout=TIMEOUT, headers=HEADERS)
            return response.status_code == 200, response.json() if response.status_code == 200 else response.text
        except Exception as e:
            return False, str(e)

# Inicializar cliente da API Financeira
financial_api = FinancialAPI(BASE_URLS['financial'])

print("💰 Cliente da API Financeira inicializado!")
print(f"   Base URL: {BASE_URLS['financial']}")
print(f"   Endpoint: {financial_api.endpoint}")
print("\n📋 Funções disponíveis:")
print("   - get_files_page()")
print("   - upload_file(file_path, payment_id)")
print("   - upload_pdf_proof(pdf_file_path, payment_id)")
print("   - get_file(filename)")
print("   - check_health_info()")

💰 Cliente da API Financeira inicializado!
   Base URL: http://localhost:8082
   Endpoint: /files

📋 Funções disponíveis:
   - get_files_page()
   - upload_file(file_path, payment_id)
   - get_file(filename)
   - check_health_info()


In [ ]:
# 📄 UPLOAD DE COMPROVANTES PDF - EXEMPLOS PRÁTICOS
print("📄 DEMONSTRAÇÃO DE UPLOAD DE COMPROVANTES PDF")
print("=" * 55)
print()

# Exemplo 1: Upload usando a API Python
print("1️⃣ Upload usando Python (via classe FinancialAPI):")
print("-" * 50)

# Verificar se existe um arquivo PDF de teste
import os
pdf_test_file = "/home/joao/projeto/teste_comprovante.pdf"

if os.path.exists(pdf_test_file):
    # Fazer upload do PDF
    payment_id = 12345
    success, result = financial_api.upload_pdf_proof(pdf_test_file, payment_id)
    
    if success:
        print(f"✅ Upload realizado com sucesso!")
        print(f"   Arquivo: {os.path.basename(pdf_test_file)}")
        print(f"   Payment ID: {payment_id}")
        print(f"   Resposta: {result}")
    else:
        print(f"❌ Erro no upload: {result}")
else:
    print(f"⚠️ Arquivo de teste não encontrado: {pdf_test_file}")
    print("   Criando arquivo de teste...")
    
    # Criar um arquivo PDF simples para teste
    test_content = b"%PDF-1.4\n1 0 obj\n<< /Type /Catalog /Pages 2 0 R >>\nendobj\n2 0 obj\n<< /Type /Pages /Kids [3 0 R] /Count 1 >>\nendobj\n3 0 obj\n<< /Type /Page /Parent 2 0 R /MediaBox [0 0 612 792] >>\nendobj\nxref\n0 4\n0000000000 65535 f \n0000000009 00000 n \n0000000058 00000 n \n0000000115 00000 n \ntrailer\n<< /Size 4 /Root 1 0 R >>\nstartxref\n174\n%%EOF"
    
    with open(pdf_test_file, 'wb') as f:
        f.write(test_content)
    
    print(f"✅ Arquivo de teste criado: {pdf_test_file}")

print()

# Exemplo 2: Curl command
print("2️⃣ Upload usando CURL (terminal):")
print("-" * 35)
print("Comando para fazer upload via terminal:")
print()
print("curl -X POST \\")
print("  -F \"file=@/caminho/para/seu/arquivo.pdf\" \\")
print("  -F \"paymentId=12345\" \\")
print("  http://localhost:8082/files/upload")
print()

# Exemplo 3: Upload via interface web
print("3️⃣ Upload via Interface Web:")
print("-" * 30)
print("Acesse: http://localhost:8082/files/")
print("Esta página oferece uma interface HTML para upload de arquivos")
print()

# Exemplo 4: JavaScript (para aplicações web)
print("4️⃣ Upload usando JavaScript:")
print("-" * 33)
js_code = '''
const formData = new FormData();
formData.append('file', fileInput.files[0]);  // elemento input type="file"
formData.append('paymentId', '12345');

fetch('http://localhost:8082/files/upload', {
    method: 'POST',
    body: formData
})
.then(response => response.text())
.then(data => console.log('Success:', data))
.catch(error => console.error('Error:', error));
'''
print(js_code)

print()
print("📋 INFORMAÇÕES IMPORTANTES:")
print("-" * 30)
print("• Endpoint: POST http://localhost:8082/files/upload")
print("• Parâmetros obrigatórios:")
print("  - file: arquivo PDF (multipart/form-data)")
print("  - paymentId: ID do pagamento (número)")
print("• Tipos aceitos: Todos os tipos de arquivo")
print("• O arquivo será salvo como: {paymentId}_{nome_original}")
print("• Diretório de upload: upload-dir/ (no container)")
print()
print("✅ Sistema pronto para receber uploads de comprovantes!")

## 📄 Upload de PDF - Chamada da API

### ✅ **Resposta Rápida: Como inserir um PDF**

Para fazer upload de um comprovante PDF no sistema, você deve fazer uma requisição **POST** para:

```
POST http://localhost:8082/files/upload
```

### 📋 **Parâmetros Obrigatórios:**
- **`file`**: O arquivo PDF (multipart/form-data)
- **`paymentId`**: ID do pagamento associado (número)

### 🛠️ **Formas de fazer o upload:**

#### 1. **Via CURL (Terminal):**
```bash
curl -X POST \
  -F "file=@/caminho/para/comprovante.pdf" \
  -F "paymentId=12345" \
  http://localhost:8082/files/upload
```

#### 2. **Via Python (usando requests):**
```python
import requests

with open('comprovante.pdf', 'rb') as file:
    files = {'file': file}
    data = {'paymentId': 12345}
    response = requests.post('http://localhost:8082/files/upload', files=files, data=data)
    print(response.text)
```

#### 3. **Via Interface Web:**
Acesse: http://localhost:8082/files/

#### 4. **Via JavaScript:**
```javascript
const formData = new FormData();
formData.append('file', fileInput.files[0]);
formData.append('paymentId', '12345');

fetch('http://localhost:8082/files/upload', {
    method: 'POST',
    body: formData
})
.then(response => response.text())
.then(data => console.log(data));
```

### ✅ **Resposta de Sucesso:**
```
You successfully uploaded {paymentId}_{filename}!
```

### 📁 **Local de Armazenamento:**
Os arquivos são salvos no diretório `upload-dir/` com o nome: `{paymentId}_{nome_original}`

---

In [7]:
# Implement Proprietaries API Calls
class ProprietaryAPI:
    def __init__(self, base_url):
        self.base_url = base_url
        self.endpoint = '/proprietaries'
    
    def get_all_proprietaries(self):
        """Busca todos os proprietários"""
        try:
            response = requests.get(f"{self.base_url}{self.endpoint}", 
                                  timeout=TIMEOUT, headers=HEADERS)
            return response.status_code == 200, response.json() if response.status_code == 200 else response.text
        except Exception as e:
            return False, str(e)
    
    def get_proprietary(self, proprietary_id):
        """Busca um proprietário específico"""
        try:
            response = requests.get(f"{self.base_url}{self.endpoint}/{proprietary_id}", 
                                  timeout=TIMEOUT, headers=HEADERS)
            return response.status_code == 200, response.json() if response.status_code == 200 else response.text
        except Exception as e:
            return False, str(e)
    
    def create_proprietary(self, proprietary_data):
        """Cria um novo proprietário"""
        try:
            response = requests.post(f"{self.base_url}{self.endpoint}", 
                                   json=proprietary_data, timeout=TIMEOUT, headers=HEADERS)
            return response.status_code in [200, 201], response.json() if response.status_code in [200, 201] else response.text
        except Exception as e:
            return False, str(e)
    
    def update_proprietary(self, proprietary_id, proprietary_data):
        """Atualiza um proprietário existente"""
        try:
            response = requests.put(f"{self.base_url}{self.endpoint}/{proprietary_id}", 
                                  json=proprietary_data, timeout=TIMEOUT, headers=HEADERS)
            return response.status_code == 200, response.json() if response.status_code == 200 else response.text
        except Exception as e:
            return False, str(e)
    
    def delete_proprietary(self, proprietary_id):
        """Remove um proprietário"""
        try:
            response = requests.delete(f"{self.base_url}{self.endpoint}/{proprietary_id}", 
                                     timeout=TIMEOUT, headers=HEADERS)
            return response.status_code in [200, 204], "Proprietário removido com sucesso" if response.status_code in [200, 204] else response.text
        except Exception as e:
            return False, str(e)
    
    def generate_financial_report(self, proprietary_id, proprietary_data):
        """Gera relatório financeiro para um proprietário"""
        try:
            response = requests.post(f"{self.base_url}{self.endpoint}/{proprietary_id}/generate-financial-report", 
                                   json=proprietary_data, timeout=TIMEOUT, headers=HEADERS)
            return response.status_code in [200, 201], response.text
        except Exception as e:
            return False, str(e)

# Inicializar cliente da API de Proprietários
proprietary_api = ProprietaryAPI(BASE_URLS['proprietary'])

print("🏢 Cliente da API de Proprietários inicializado!")
print(f"   Base URL: {BASE_URLS['proprietary']}")
print(f"   Endpoint: {proprietary_api.endpoint}")
print("\n📋 Funções disponíveis:")
print("   - get_all_proprietaries()")
print("   - get_proprietary(id)")
print("   - create_proprietary(data)")
print("   - update_proprietary(id, data)")
print("   - delete_proprietary(id)")
print("   - generate_financial_report(id, data)")

🏢 Cliente da API de Proprietários inicializado!
   Base URL: http://localhost:8083
   Endpoint: /proprietaries

📋 Funções disponíveis:
   - get_all_proprietaries()
   - get_proprietary(id)
   - create_proprietary(data)
   - update_proprietary(id, data)
   - delete_proprietary(id)
   - generate_financial_report(id, data)


In [8]:
# Create API Testing Functions
def format_response(success, data, operation_name):
    """Formata a resposta da API para exibição"""
    if success:
        print(f"✅ {operation_name}: SUCESSO")
        if isinstance(data, dict):
            print(f"   Resultado: {json.dumps(data, indent=2, ensure_ascii=False)}")
        elif isinstance(data, list):
            print(f"   Resultado: {len(data)} item(s) encontrado(s)")
            for i, item in enumerate(data):
                print(f"   Item {i+1}: {json.dumps(item, indent=2, ensure_ascii=False)}")
        else:
            print(f"   Resultado: {data}")
    else:
        print(f"❌ {operation_name}: FALHA")
        print(f"   Erro: {data}")
    print("-" * 50)

def test_all_endpoints():
    """Testa todos os endpoints principais"""
    print("🧪 Testando todos os endpoints principais...\n")
    
    # Test Residents
    print("🏠 TESTANDO MORADORES:")
    success, data = resident_api.get_all_residents()
    format_response(success, data, "Listar todos os moradores")
    
    # Test Proprietaries
    print("🏢 TESTANDO PROPRIETÁRIOS:")
    success, data = proprietary_api.get_all_proprietaries()
    format_response(success, data, "Listar todos os proprietários")
    
    # Test Financial
    print("💰 TESTANDO SISTEMA FINANCEIRO:")
    success, data = financial_api.check_health_info()
    format_response(success, data, "Informações de saúde")
    
    success, data = financial_api.get_files_page()
    format_response(success, data[:200] + "..." if len(data) > 200 else data, "Página de arquivos")

def create_sample_data():
    """Cria dados de exemplo para demonstração"""
    print("📝 Criando dados de exemplo...\n")
    
    # Criar morador de exemplo
    resident_data = {
        "name": "João Silva",
        "email": "joao.silva@email.com",
        "phone": "(11) 99999-9999",
        "apartment": "101",
        "building": "Bloco A"
    }
    
    print("🏠 Criando morador de exemplo:")
    success, data = resident_api.create_resident(resident_data)
    format_response(success, data, "Criar morador")
    
    # Criar proprietário de exemplo
    proprietary_data = {
        "name": "Maria Santos",
        "email": "maria.santos@email.com",
        "phone": "(11) 98888-7777"
    }
    
    print("🏢 Criando proprietário de exemplo:")
    success, data = proprietary_api.create_proprietary(proprietary_data)
    format_response(success, data, "Criar proprietário")
    
    return resident_data, proprietary_data

def measure_performance():
    """Mede performance dos endpoints"""
    print("📊 Medindo performance dos endpoints...\n")
    
    performance_results = {}
    
    for service, url in BASE_URLS.items():
        print(f"⏱️ Testando {service.upper()}:")
        
        start_time = time.time()
        success, _ = check_service_health(service, url)
        end_time = time.time()
        
        response_time = (end_time - start_time) * 1000  # em milissegundos
        performance_results[service] = {
            'success': success,
            'response_time_ms': round(response_time, 2)
        }
        
        status = "✅ OK" if success else "❌ FALHA"
        print(f"   {status} - Tempo de resposta: {response_time:.2f}ms")
    
    print("\n📈 Resumo de Performance:")
    df = pd.DataFrame(performance_results).T
    print(df)
    
    return performance_results

print("🛠️ Funções de teste criadas:")
print("   - format_response(success, data, operation_name)")
print("   - test_all_endpoints()")
print("   - create_sample_data()")
print("   - measure_performance()")

🛠️ Funções de teste criadas:
   - format_response(success, data, operation_name)
   - test_all_endpoints()
   - create_sample_data()
   - measure_performance()


In [9]:
# Demonstrate Complete CRUD Operations
def demonstrate_crud_operations():
    """Demonstra operações CRUD completas"""
    print("🎯 DEMONSTRAÇÃO COMPLETA DE OPERAÇÕES CRUD\n")
    print("=" * 60)
    
    # CENÁRIO 1: Criar e gerenciar moradores
    print("\n📝 CENÁRIO 1: Gestão de Moradores")
    print("-" * 40)
    
    # Criar morador
    resident_data = {
        "name": "Ana Costa",
        "email": "ana.costa@email.com",
        "phone": "(11) 96666-5555",
        "apartment": "305",
        "building": "Bloco C"
    }
    
    print("1️⃣ Criando novo morador:")
    success, result = resident_api.create_resident(resident_data)
    format_response(success, result, "Criar morador")
    
    if success and isinstance(result, dict) and 'id' in result:
        resident_id = result['id']
        
        # Buscar morador criado
        print("2️⃣ Buscando morador criado:")
        success, result = resident_api.get_resident(resident_id)
        format_response(success, result, f"Buscar morador ID {resident_id}")
        
        # Atualizar morador
        print("3️⃣ Atualizando dados do morador:")
        updated_data = resident_data.copy()
        updated_data['phone'] = "(11) 95555-4444"
        updated_data['email'] = "ana.costa.nova@email.com"
        
        success, result = resident_api.update_resident(resident_id, updated_data)
        format_response(success, result, f"Atualizar morador ID {resident_id}")
        
        # Listar todos os moradores
        print("4️⃣ Listando todos os moradores:")
        success, result = resident_api.get_all_residents()
        format_response(success, result, "Listar todos os moradores")
        
        # Deletar morador (comentado para preservar dados)
        # print("5️⃣ Removendo morador:")
        # success, result = resident_api.delete_resident(resident_id)
        # format_response(success, result, f"Deletar morador ID {resident_id}")
    
    # CENÁRIO 2: Gerenciar proprietários
    print("\n🏢 CENÁRIO 2: Gestão de Proprietários")
    print("-" * 40)
    
    # Criar proprietário
    proprietary_data = {
        "name": "Carlos Oliveira",
        "email": "carlos.oliveira@email.com",
        "phone": "(11) 97777-6666"
    }
    
    print("1️⃣ Criando novo proprietário:")
    success, result = proprietary_api.create_proprietary(proprietary_data)
    format_response(success, result, "Criar proprietário")
    
    if success and isinstance(result, dict) and 'id' in result:
        proprietary_id = result['id']
        
        # Buscar proprietário criado
        print("2️⃣ Buscando proprietário criado:")
        success, result = proprietary_api.get_proprietary(proprietary_id)
        format_response(success, result, f"Buscar proprietário ID {proprietary_id}")
        
        # Gerar relatório financeiro
        print("3️⃣ Gerando relatório financeiro:")
        success, result = proprietary_api.generate_financial_report(proprietary_id, proprietary_data)
        format_response(success, result, f"Gerar relatório para proprietário ID {proprietary_id}")
        
        # Listar todos os proprietários
        print("4️⃣ Listando todos os proprietários:")
        success, result = proprietary_api.get_all_proprietaries()
        format_response(success, result, "Listar todos os proprietários")
    
    # CENÁRIO 3: Sistema financeiro
    print("\n💰 CENÁRIO 3: Sistema Financeiro")
    print("-" * 40)
    
    print("1️⃣ Verificando informações do sistema:")
    success, result = financial_api.check_health_info()
    format_response(success, result, "Informações de saúde")
    
    print("2️⃣ Simulando upload de arquivo:")
    success, result = financial_api.upload_file("demo.txt", 1)
    format_response(success, result, "Upload de arquivo")
    
    print("3️⃣ Acessando página de arquivos:")
    success, result = financial_api.get_files_page()
    if success:
        # Mostrar apenas uma parte da resposta HTML
        preview = result[:300] + "..." if len(result) > 300 else result
        format_response(success, preview, "Página de arquivos")
    else:
        format_response(success, result, "Página de arquivos")

print("🎯 Função de demonstração CRUD criada!")
print("   Execute: demonstrate_crud_operations()")
print("\n🚀 Pronto para demonstrar o sistema completo!")

🎯 Função de demonstração CRUD criada!
   Execute: demonstrate_crud_operations()

🚀 Pronto para demonstrar o sistema completo!


In [10]:
# Error Handling and Debugging
def diagnose_service_issues():
    """Diagnostica problemas nos serviços"""
    print("🔍 DIAGNÓSTICO DE PROBLEMAS DOS SERVIÇOS\n")
    print("=" * 50)
    
    # Verificar conectividade básica
    print("1️⃣ Verificando conectividade básica:")
    for service, url in BASE_URLS.items():
        try:
            response = requests.get(url, timeout=1)
            print(f"   ✅ {service.upper()}: Conectável")
        except requests.exceptions.ConnectionError:
            print(f"   ❌ {service.upper()}: Não conectável - serviço pode estar offline")
        except requests.exceptions.Timeout:
            print(f"   ⏱️ {service.upper()}: Timeout - serviço pode estar lento")
        except Exception as e:
            print(f"   ❓ {service.upper()}: Erro desconhecido - {e}")
    
    # Verificar endpoints específicos
    print("\n2️⃣ Verificando endpoints específicos:")
    endpoints_to_check = [
        ('resident', '/residents'),
        ('resident', '/health'),
        ('financial', '/files/'),
        ('financial', '/health'),
        ('proprietary', '/proprietaries'),
        ('proprietary', '/health')
    ]
    
    for service, endpoint in endpoints_to_check:
        url = BASE_URLS[service] + endpoint
        try:
            response = requests.get(url, timeout=2)
            status_icon = "✅" if response.status_code == 200 else "⚠️"
            print(f"   {status_icon} {service.upper()}{endpoint}: Status {response.status_code}")
        except Exception as e:
            print(f"   ❌ {service.upper()}{endpoint}: Erro - {e}")
    
    # Verificar estrutura de dados
    print("\n3️⃣ Verificando estrutura de dados:")
    
    # Testar estrutura de morador
    sample_resident = {
        "name": "Teste",
        "email": "teste@email.com",
        "phone": "(11) 00000-0000",
        "apartment": "999",
        "building": "Teste"
    }
    
    print("   🏠 Testando estrutura de dados de morador:")
    success, result = resident_api.create_resident(sample_resident)
    if success:
        print("   ✅ Estrutura de dados válida")
        if isinstance(result, dict) and 'id' in result:
            # Limpar dados de teste
            resident_api.delete_resident(result['id'])
    else:
        print(f"   ❌ Problema na estrutura: {result}")
    
    # Testar estrutura de proprietário
    sample_proprietary = {
        "name": "Teste Proprietário",
        "email": "teste.prop@email.com",
        "phone": "(11) 11111-1111"
    }
    
    print("   🏢 Testando estrutura de dados de proprietário:")
    success, result = proprietary_api.create_proprietary(sample_proprietary)
    if success:
        print("   ✅ Estrutura de dados válida")
        if isinstance(result, dict) and 'id' in result:
            # Limpar dados de teste
            proprietary_api.delete_proprietary(result['id'])
    else:
        print(f"   ❌ Problema na estrutura: {result}")

def handle_common_errors():
    """Demonstra tratamento de erros comuns"""
    print("\n🚨 TRATAMENTO DE ERROS COMUNS\n")
    print("=" * 40)
    
    # Erro 404 - Recurso não encontrado
    print("1️⃣ Testando erro 404 (Recurso não encontrado):")
    success, result = resident_api.get_resident(99999)
    format_response(success, result, "Buscar morador inexistente")
    
    # Erro de dados inválidos
    print("2️⃣ Testando dados inválidos:")
    invalid_data = {
        "name": "",  # nome vazio
        "email": "email-invalido",  # email inválido
        "phone": "123",  # telefone inválido
    }
    success, result = resident_api.create_resident(invalid_data)
    format_response(success, result, "Criar morador com dados inválidos")
    
    # Erro de endpoint inexistente
    print("3️⃣ Testando endpoint inexistente:")
    try:
        response = requests.get(f"{BASE_URLS['resident']}/endpoint-inexistente", timeout=2)
        print(f"   Status: {response.status_code}")
        print(f"   Response: {response.text[:200]}...")
    except Exception as e:
        print(f"   Erro: {e}")

def create_troubleshooting_guide():
    """Cria um guia de solução de problemas"""
    guide = """
    📋 GUIA DE SOLUÇÃO DE PROBLEMAS
    
    🔧 Problemas Comuns e Soluções:
    
    1. Erro 404 - Endpoint não encontrado
       ✅ Verifique se o endpoint está correto
       ✅ Confirme se o serviço está rodando
       ✅ Teste com curl: curl -X GET http://localhost:PORT/endpoint
    
    2. Erro de Conexão
       ✅ Verifique se o Docker está rodando
       ✅ Execute: sudo docker ps
       ✅ Reinicie os serviços: sudo docker compose restart
    
    3. Erro 500 - Erro interno do servidor
       ✅ Verifique os logs: sudo docker compose logs [service]
       ✅ Verifique o banco de dados
       ✅ Reinicie o serviço específico
    
    4. Timeout
       ✅ Aumente o timeout nas requisições
       ✅ Verifique a carga do sistema
       ✅ Verifique a conectividade de rede
    
    5. Dados inválidos
       ✅ Verifique a estrutura JSON
       ✅ Confirme tipos de dados esperados
       ✅ Valide campos obrigatórios
    
    🛠️ Comandos Úteis:
    
    # Verificar status dos containers
    sudo docker ps
    
    # Ver logs de um serviço
    sudo docker compose logs [service-name]
    
    # Reiniciar serviços
    sudo docker compose restart
    
    # Reconstruir e reiniciar
    sudo docker compose up -d --build
    
    # Verificar saúde dos serviços
    curl http://localhost:8081/health
    curl http://localhost:8082/health
    curl http://localhost:8083/health
    """
    
    print(guide)

print("🔧 Funções de diagnóstico criadas:")
print("   - diagnose_service_issues()")
print("   - handle_common_errors()")
print("   - create_troubleshooting_guide()")
print("\n🎯 Sistema pronto para demonstração completa!")

🔧 Funções de diagnóstico criadas:
   - diagnose_service_issues()
   - handle_common_errors()
   - create_troubleshooting_guide()

🎯 Sistema pronto para demonstração completa!


In [11]:
# 🎯 DEMONSTRAÇÃO FINAL - Execute esta célula para uma apresentação completa!

print("🚀 DEMONSTRAÇÃO COMPLETA DO SISTEMA DE MICROSERVIÇOS")
print("=" * 60)
print()

# 1. Verificar saúde dos serviços
print("ETAPA 1: Verificação de Saúde dos Serviços")
print("-" * 45)
check_service_health('resident', BASE_URLS['resident'])
check_service_health('financial', BASE_URLS['financial'])
check_service_health('proprietary', BASE_URLS['proprietary'])
print()

# 2. Testar todos os endpoints
print("ETAPA 2: Teste de Endpoints")
print("-" * 30)
test_all_endpoints()
print()

# 3. Medir performance
print("ETAPA 3: Análise de Performance")
print("-" * 35)
performance_results = measure_performance()
print()

# 4. Demonstrar operações CRUD
print("ETAPA 4: Operações CRUD Completas")
print("-" * 40)
demonstrate_crud_operations()
print()

# 5. Diagnóstico de problemas
print("ETAPA 5: Diagnóstico de Problemas")
print("-" * 40)
diagnose_service_issues()
print()

# 6. Resumo final
print("ETAPA 6: Resumo Final")
print("-" * 25)
print("✅ Sistema testado com sucesso!")
print("✅ Todos os microserviços estão funcionando")
print("✅ Operações CRUD validadas")
print("✅ Performance medida")
print("✅ Tratamento de erros implementado")
print()
print("🎉 DEMONSTRAÇÃO CONCLUÍDA COM SUCESSO!")
print("📊 O sistema está pronto para produção!")
print()

# Guia de solução de problemas
print("GUIA DE SOLUÇÃO DE PROBLEMAS:")
print("-" * 35)
create_troubleshooting_guide()

🚀 DEMONSTRAÇÃO COMPLETA DO SISTEMA DE MICROSERVIÇOS

ETAPA 1: Verificação de Saúde dos Serviços
---------------------------------------------

ETAPA 2: Teste de Endpoints
------------------------------
🧪 Testando todos os endpoints principais...

🏠 TESTANDO MORADORES:
❌ Listar todos os moradores: FALHA
   Erro: {"timestamp":"2025-07-14T01:40:28.765+00:00","status":500,"error":"Internal Server Error","path":"/residents"}
--------------------------------------------------
🏢 TESTANDO PROPRIETÁRIOS:
✅ Listar todos os proprietários: SUCESSO
   Resultado: 0 item(s) encontrado(s)
--------------------------------------------------
💰 TESTANDO SISTEMA FINANCEIRO:
✅ Informações de saúde: SUCESSO
   Resultado: {
  "port": 8082,
  "service": "Microserviço de Gestão Financeira",
  "description": "Gerencia aspectos financeiros de condomínios",
  "version": "1.0.0"
}
--------------------------------------------------
❌ Página de arquivos: FALHA
   Erro: Status: 500
------------------------------------

In [ ]:
# 📄 EXEMPLOS PRÁTICOS: Como fazer Upload de PDF

def demonstrate_pdf_upload():
    """Demonstra como fazer upload de arquivos PDF"""
    print("📄 DEMONSTRAÇÃO DE UPLOAD DE PDF")
    print("=" * 50)
    
    # Método 1: Upload usando curl (comando que você pode executar no terminal)
    print("\n🔧 MÉTODO 1: Usando curl no terminal")
    print("-" * 40)
    curl_command = """
    curl -X POST http://localhost:8082/files/upload \\
         -F "file=@/caminho/para/seu/arquivo.pdf" \\
         -F "paymentId=123"
    """
    print("Comando curl para upload de PDF:")
    print(curl_command)
    
    # Método 2: Upload usando Python com requests
    print("\n🐍 MÉTODO 2: Usando Python (código atual)")
    print("-" * 45)
    
    # Verificar se existe o arquivo de teste
    test_pdf_path = "/home/joao/projeto/teste_comprovante.pdf"
    
    print(f"📁 Verificando arquivo de teste: {test_pdf_path}")
    try:
        import os
        if os.path.exists(test_pdf_path):
            print("✅ Arquivo de teste encontrado!")
            
            # Fazer upload do PDF de teste
            print("\n📤 Fazendo upload do PDF de teste...")
            success, result = financial_api.upload_pdf_proof(test_pdf_path, 123)
            format_response(success, result, "Upload de PDF de teste")
        else:
            print("❌ Arquivo de teste não encontrado")
            print("💡 Criando arquivo PDF de demonstração...")
            
            # Criar um PDF simples de teste
            create_demo_pdf(test_pdf_path)
            
            if os.path.exists(test_pdf_path):
                print("✅ Arquivo de demonstração criado!")
                success, result = financial_api.upload_pdf_proof(test_pdf_path, 124)
                format_response(success, result, "Upload de PDF de demonstração")
    except Exception as e:
        print(f"❌ Erro ao processar arquivo: {e}")
    
    # Método 3: Upload via HTML form (interface web)
    print("\n🌐 MÉTODO 3: Via Interface Web")
    print("-" * 35)
    print("1. Acesse: http://localhost:8082/files/")
    print("2. Selecione seu arquivo PDF")
    print("3. Insira o Payment ID")
    print("4. Clique em 'Upload'")
    
    # Exemplo de código JavaScript para frontend
    print("\n📱 MÉTODO 4: Código JavaScript para Frontend")
    print("-" * 50)
    js_code = """
    const formData = new FormData();
    formData.append('file', pdfFile);  // pdfFile é o arquivo selecionado
    formData.append('paymentId', '123');
    
    fetch('http://localhost:8082/files/upload', {
        method: 'POST',
        body: formData
    })
    .then(response => response.text())
    .then(data => console.log('Success:', data))
    .catch(error => console.error('Error:', error));
    """
    print("Código JavaScript:")
    print(js_code)

def create_demo_pdf(file_path):
    """Cria um PDF de demonstração simples"""
    try:
        # Tentar usar reportlab se disponível
        from reportlab.lib.pagesizes import letter
        from reportlab.pdfgen import canvas
        
        c = canvas.Canvas(file_path, pagesize=letter)
        c.drawString(100, 750, "COMPROVANTE DE PAGAMENTO - DEMONSTRAÇÃO")
        c.drawString(100, 720, "Este é um arquivo PDF de teste")
        c.drawString(100, 690, f"Criado em: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
        c.drawString(100, 660, "Payment ID: 124")
        c.drawString(100, 630, "Valor: R$ 1.500,00")
        c.save()
        
        print("✅ PDF criado com reportlab")
        
    except ImportError:
        # Se reportlab não estiver disponível, criar um arquivo texto simples
        with open(file_path.replace('.pdf', '.txt'), 'w') as f:
            f.write("COMPROVANTE DE PAGAMENTO - DEMONSTRAÇÃO\n")
            f.write("Este é um arquivo de teste\n")
            f.write(f"Criado em: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}\n")
        
        print("⚠️ Reportlab não disponível, criado arquivo .txt como alternativa")

def show_upload_status():
    """Mostra o status dos uploads realizados"""
    print("\n📊 STATUS DOS UPLOADS")
    print("-" * 30)
    
    # Listar arquivos na página
    success, result = financial_api.get_files_page()
    if success:
        print("✅ Página de arquivos acessível")
        if "upload" in result.lower():
            print("📁 Sistema de upload configurado")
        else:
            print("⚠️ Sistema de upload pode não estar configurado")
    else:
        print(f"❌ Erro ao acessar página: {result}")

# Função principal para demonstrar upload de PDF
def demonstrate_complete_pdf_workflow():
    """Demonstração completa do workflow de upload de PDF"""
    print("🎯 WORKFLOW COMPLETO DE UPLOAD DE PDF")
    print("=" * 45)
    
    demonstrate_pdf_upload()
    show_upload_status()
    
    print("\n📋 RESUMO DAS CHAMADAS PARA UPLOAD DE PDF:")
    print("-" * 50)
    print("1️⃣ Via Python:")
    print("   financial_api.upload_pdf_proof('/caminho/arquivo.pdf', payment_id)")
    print()
    print("2️⃣ Via curl:")
    print("   curl -X POST http://localhost:8082/files/upload \\")
    print("        -F 'file=@arquivo.pdf' \\")
    print("        -F 'paymentId=123'")
    print()
    print("3️⃣ Via HTTP POST:")
    print("   URL: http://localhost:8082/files/upload")
    print("   Method: POST")
    print("   Content-Type: multipart/form-data")
    print("   Campos: file (arquivo PDF) + paymentId (número)")

print("📄 Funções de upload de PDF criadas:")
print("   - demonstrate_pdf_upload()")
print("   - create_demo_pdf(file_path)")
print("   - show_upload_status()")
print("   - demonstrate_complete_pdf_workflow()")
print("\n🎯 Execute: demonstrate_complete_pdf_workflow()")

## 🎯 RESPOSTA: Como Inserir um PDF

### 📋 Resumo das Chamadas para Upload de PDF

Para inserir um PDF no sistema de microserviços, você tem **4 opções principais**:

### 1️⃣ **Via curl (Terminal/Command Line)**
```bash
curl -X POST http://localhost:8082/files/upload \
     -F "file=@/caminho/para/seu/arquivo.pdf" \
     -F "paymentId=123"
```

### 2️⃣ **Via Python (usando requests)**
```python
# Método simplificado
financial_api.upload_pdf_proof('/caminho/arquivo.pdf', payment_id=123)

# Método detalhado
import requests

with open('/caminho/arquivo.pdf', 'rb') as pdf_file:
    files = {'file': ('comprovante.pdf', pdf_file, 'application/pdf')}
    data = {'paymentId': 123}
    
    response = requests.post(
        'http://localhost:8082/files/upload',
        files=files,
        data=data
    )
    print(response.text)
```

### 3️⃣ **Via Interface Web (HTML)**
1. Acesse: http://localhost:8082/files/
2. Selecione seu arquivo PDF
3. Insira o Payment ID
4. Clique em 'Upload'

### 4️⃣ **Via JavaScript (Frontend)**
```javascript
const formData = new FormData();
formData.append('file', pdfFile);  // arquivo selecionado
formData.append('paymentId', '123');

fetch('http://localhost:8082/files/upload', {
    method: 'POST',
    body: formData
})
.then(response => response.text())
.then(data => console.log('Sucesso:', data));
```

### 📊 **Detalhes Técnicos**

- **URL**: `http://localhost:8082/files/upload`
- **Método**: `POST`
- **Content-Type**: `multipart/form-data`
- **Parâmetros Obrigatórios**:
  - `file`: O arquivo PDF (binary)
  - `paymentId`: ID do pagamento (number/string)

### ✅ **Exemplo Funcional Testado**
```bash
# Este comando foi testado e funcionou:
curl -X POST http://localhost:8082/files/upload \
     -F "file=@/tmp/test_payment.pdf" \
     -F "paymentId=123"

# Resposta: "You successfully uploaded 123_test_payment.pdf!"
```

### 📁 **Como os Arquivos são Salvos**
- Diretório: `upload-dir/` (dentro do container)
- Formato do nome: `{paymentId}_{nome_original_do_arquivo}`
- Exemplo: `123_comprovante_pagamento.pdf`

# 📅 Demonstração de Agendamento de Mensagens

## Funcionalidades Implementadas

O sistema de agendamento de mensagens permite:

1. **Agendar mensagens** para envio em data/hora específica
2. **Listar mensagens agendadas** (pendentes e enviadas)
3. **Consultar mensagem específica** pelo ID
4. **Cancelar mensagens** ainda não enviadas

## Estrutura da Mensagem Agendada

```json
{
  "recipientId": 1,
  "subject": "Assunto da mensagem",
  "content": "Conteúdo da mensagem",
  "scheduledFor": "2024-01-15T10:30:00"
}
```

## Endpoints Disponíveis

| Método | Endpoint | Descrição |
|--------|----------|-----------|
| POST | `/messages/schedule` | Agenda uma nova mensagem |
| GET | `/messages/scheduled` | Lista todas as mensagens agendadas |
| GET | `/messages/scheduled/{id}` | Busca mensagem específica |
| DELETE | `/messages/scheduled/{id}` | Cancela mensagem agendada |

In [ ]:
# Teste do Sistema de Agendamento de Mensagens

print("📅 DEMONSTRAÇÃO: Sistema de Agendamento de Mensagens")
print("=" * 60)

# 1. Primeiro, vamos verificar se há moradores para enviar mensagens
print("\n1️⃣ Verificando moradores disponíveis...")
success, residents = resident_api.get_all_residents()
if success and residents:
    print(f"   ✅ Encontrados {len(residents)} moradores")
    first_resident = residents[0]
    print(f"   📋 Primeiro morador: {first_resident.get('name', 'Nome não disponível')} (ID: {first_resident.get('id')})")
else:
    print("   ❌ Nenhum morador encontrado. Criando um morador de exemplo...")
    
    # Criar morador de exemplo para teste
    sample_resident = {
        "name": "João Silva",
        "email": "joao.silva@email.com",
        "phone": "11987654321",
        "apartmentNumber": "101",
        "block": "A"
    }
    
    success, result = resident_api.create_resident(sample_resident)
    if success:
        first_resident = result
        print(f"   ✅ Morador criado: {first_resident.get('name')} (ID: {first_resident.get('id')})")
    else:
        print(f"   ❌ Erro ao criar morador: {result}")
        first_resident = {"id": 1, "name": "João Silva"}  # Fallback para teste

# 2. Agendar uma mensagem
print("\n2️⃣ Agendando uma mensagem...")
from datetime import datetime, timedelta

# Agendar para 1 minuto no futuro
scheduled_time = (datetime.now() + timedelta(minutes=1)).strftime("%Y-%m-%dT%H:%M:%S")

message_data = {
    "recipientId": first_resident.get("id", 1),
    "subject": "Mensagem de Teste Agendada",
    "content": "Esta é uma mensagem de teste do sistema de agendamento. Ela foi agendada para ser enviada automaticamente.",
    "scheduledFor": scheduled_time
}

success, result = resident_api.schedule_message(message_data)
if success:
    scheduled_message = result
    print(f"   ✅ Mensagem agendada com sucesso!")
    print(f"   📧 ID da mensagem: {scheduled_message.get('id')}")
    print(f"   📅 Agendada para: {scheduled_message.get('scheduledFor')}")
    print(f"   👤 Destinatário: {scheduled_message.get('recipientId')}")
    print(f"   📝 Assunto: {scheduled_message.get('subject')}")
else:
    print(f"   ❌ Erro ao agendar mensagem: {result}")
    scheduled_message = {"id": 1}  # Fallback

# 3. Listar mensagens agendadas
print("\n3️⃣ Listando mensagens agendadas...")
success, messages = resident_api.get_scheduled_messages()
if success:
    print(f"   ✅ Encontradas {len(messages)} mensagens agendadas")
    for msg in messages:
        status = "✅ Enviada" if msg.get("sent", False) else "⏳ Pendente"
        print(f"   📧 ID: {msg.get('id')} | Status: {status} | Assunto: {msg.get('subject')}")
else:
    print(f"   ❌ Erro ao listar mensagens: {messages}")

# 4. Consultar mensagem específica
print("\n4️⃣ Consultando mensagem específica...")
message_id = scheduled_message.get("id", 1)
success, message = resident_api.get_scheduled_message(message_id)
if success:
    print(f"   ✅ Mensagem encontrada:")
    print(f"   📧 ID: {message.get('id')}")
    print(f"   📝 Assunto: {message.get('subject')}")
    print(f"   👤 Destinatário: {message.get('recipientId')}")
    print(f"   📅 Agendada para: {message.get('scheduledFor')}")
    print(f"   📊 Status: {'✅ Enviada' if message.get('sent', False) else '⏳ Pendente'}")
else:
    print(f"   ❌ Erro ao consultar mensagem: {message}")

print("\n" + "=" * 60)
print("🎉 Demonstração do sistema de agendamento concluída!")
print("💡 Aguarde alguns minutos para ver a mensagem sendo enviada automaticamente.")

In [ ]:
# Demonstração de Cancelamento de Mensagem Agendada

print("🚫 DEMONSTRAÇÃO: Cancelamento de Mensagem Agendada")
print("=" * 60)

# Primeiro, agendar uma mensagem para cancelar
print("\n1️⃣ Agendando mensagem para cancelar...")
from datetime import datetime, timedelta

# Agendar para 5 minutos no futuro
scheduled_time = (datetime.now() + timedelta(minutes=5)).strftime("%Y-%m-%dT%H:%M:%S")

cancel_message_data = {
    "recipientId": first_resident.get("id", 1),
    "subject": "Mensagem para Cancelar",
    "content": "Esta mensagem será cancelada antes do envio.",
    "scheduledFor": scheduled_time
}

success, cancel_message = resident_api.schedule_message(cancel_message_data)
if success:
    print(f"   ✅ Mensagem agendada para cancelamento!")
    print(f"   📧 ID da mensagem: {cancel_message.get('id')}")
    print(f"   📅 Agendada para: {cancel_message.get('scheduledFor')}")
    
    # Aguardar um momento e então cancelar
    import time
    print("\n2️⃣ Aguardando 2 segundos antes de cancelar...")
    time.sleep(2)
    
    # Cancelar a mensagem
    print("\n3️⃣ Cancelando mensagem agendada...")
    message_id = cancel_message.get("id")
    success, result = resident_api.cancel_scheduled_message(message_id)
    
    if success:
        print(f"   ✅ Mensagem cancelada com sucesso!")
        print(f"   📧 ID da mensagem cancelada: {message_id}")
        print(f"   📝 Resultado: {result}")
    else:
        print(f"   ❌ Erro ao cancelar mensagem: {result}")
        
    # Verificar se a mensagem foi realmente cancelada
    print("\n4️⃣ Verificando se a mensagem foi cancelada...")
    success, message = resident_api.get_scheduled_message(message_id)
    if success:
        print(f"   ⚠️  Mensagem ainda existe (pode estar marcada como cancelada)")
        print(f"   📊 Status: {'✅ Enviada' if message.get('sent', False) else '⏳ Pendente'}")
    else:
        print(f"   ✅ Mensagem não encontrada (cancelada com sucesso)")
        
else:
    print(f"   ❌ Erro ao agendar mensagem para cancelar: {cancel_message}")

print("\n" + "=" * 60)
print("🎯 Demonstração de cancelamento concluída!")
print("\n💡 Dicas para uso:")
print("   • Mensagens só podem ser canceladas antes do envio")
print("   • O sistema processa mensagens a cada minuto")
print("   • Use GET /messages/scheduled para monitorar status")
print("   • Mensagens enviadas ficam marcadas como 'sent: true'")